
---
### 🗂️ Histórico — downloader & combinador (b3_hist_downloader.py)

**Resumo rápido**  
Script que baixa históricos de preços (via `yfinance`), salva por ticker em **parquet** (e opcionalmente CSV), e permite combinar todos os parquets em um único arquivo *long-format*. Feito para rodar em batch com retry/backoff, pular tickers já salvos e gerar um resumo final.


#### ✨ Principais responsabilidades
- Baixar históricos de preço por lotes (batch) com `yfinance`.  
- Salvar cada ticker em `data/historical/{TICKER}.parquet` e `{TICKER}.csv`.  
- Gerenciar retries (exponencial), fallback ticker-a-ticker e logs.  
- Gerar CSV de resumo por execução (`download_summary_YYYYMMDDTHHMMSSZ.csv`).  
- Recombinar todos os parquets em um único `all_histories.parquet` / `all_histories.csv`.


#### 🧩 Funções principais (one-liners)
- `ticker_exists_local(ticker)` → verifica existência de `{ticker}.parquet`.  
- `save_history_df(ticker, df, save_csv=True)` → salva parquet e CSV; garante coluna `date`.  
- `download_batch(batch, start, threads)` → tenta baixar um batch via `yfinance.download` com retries e fallback.  
- `download_all_histories(tickers, start, force, save_summary, save_csv_per_ticker)` → orquestra o download em batches, salva e retorna um `DataFrame` resumo.  
- `combine_all_to_single_parquet(out_path, out_csv, tickers)` → concatena todos os parquets em formato long e salva.


#### ⚙️ Parâmetros principais (valores padrão)
| Parâmetro | Valor padrão |
|---:|:---|
| `HIST_DIR` | `data/historical/` |
| `DEFAULT_START` | `"2011-01-01"` |
| `BATCH_SIZE` | `15` |
| `MAX_ATTEMPTS` | `4` |
| `SLEEP_BETWEEN_BATCHES` | `1` (seg) |
| `SLEEP_BETWEEN_TICKERS` | `0.2` (seg) |


#### 📂 Saídas geradas
- `data/historical/{TICKER}.parquet` — parquet por ticker.  
- `data/historical/{TICKER}.csv` —  CSV por ticker.  
- `data/historical/download_summary_{ts}.csv` — resumo da execução.  
- `data/historical/all_histories.parquet` & `all_histories.csv` — concat final (long format).

---

In [ ]:
# logging simples
import datetime
import logging
import os
import time

import pandas as pd

from typing import Dict, Iterable, List, Optional, Set

import yfinance as yf

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# diretório onde os históricos serão salvos (pasta data no nível do aurum, não data_sources)
HIST_DIR = os.path.join("..", "data", "historical")
os.makedirs(HIST_DIR, exist_ok=True)

# parâmetros de download
DEFAULT_START = "2011-01-01"
BATCH_SIZE = 15
MAX_ATTEMPTS = 4          
SLEEP_BETWEEN_BATCHES = 1  
SLEEP_BETWEEN_TICKERS = 0.2

def ticker_exists_local(ticker: str) -> bool:
    """Verifica se já existe parquet salvo para ticker (usa para pular downloads)"""
    path = os.path.join(HIST_DIR, f"{ticker}.parquet")
    return os.path.isfile(path)

def save_history_df(ticker: str, df: pd.DataFrame, save_csv: bool = True):
    """Salva DataFrame em parquet e opcionalmente em CSV. Garante coluna 'date' se índice for DatetimeIndex."""
    if df is None or df.empty:
        raise ValueError("DataFrame nulo ou vazio")
    df = df.copy()
    # garantir que a coluna de data exista como coluna
    if isinstance(df.index, pd.DatetimeIndex):
        df.index.name = "date"
        df = df.reset_index()
    # converter coluna date para string ISO ao salvar CSV (mantém compatibilidade)
    out_parquet = os.path.join(HIST_DIR, f"{ticker}.parquet")
    out_csv = os.path.join(HIST_DIR, f"{ticker}.csv")
    try:
        df.to_parquet(out_parquet, index=False)
        logging.info("Saved %s rows for %s -> %s", len(df), ticker, out_parquet)
    except Exception as e:
        logging.exception("Erro salvando parquet para %s: %s", ticker, e)
        raise
    if save_csv:
        try:
            # padronizar data para ISO antes de salvar CSV (se existir)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
            df.to_csv(out_csv, index=False)
            logging.info("Saved CSV for %s -> %s", ticker, out_csv)
        except Exception as e:
            logging.exception("Erro salvando CSV para %s: %s", ticker, e)
            # não raise — parquet já salvo, apenas logamos o problema

def download_batch(batch: List[str], start: str = DEFAULT_START, threads: bool = True) -> Dict[str, Optional[pd.DataFrame]]:
    """
    Tenta baixar um batch de tickers via yfinance.download.
    Retorna dict ticker -> DataFrame or None (se falhou).
    """
    joined = " ".join(batch)
    attempt = 0
    last_exc = None
    while attempt < MAX_ATTEMPTS:
        try:
            logging.info("yfinance.download attempt %d for batch size %d", attempt+1, len(batch))
            data = yf.download(tickers=joined, start=start, progress=False, threads=threads, group_by='ticker', auto_adjust=False, actions=True)
            result = {}
            if isinstance(data, pd.DataFrame) and isinstance(data.columns, pd.MultiIndex):
                for ticker in batch:
                    if ticker in data.columns.get_level_values(0):
                        df_t = data[ticker].copy()
                        result[ticker] = df_t
                    else:
                        try:
                            single = yf.download(ticker, start=start, progress=False, actions=True)
                            result[ticker] = single if not single.empty else None
                        except Exception:
                            result[ticker] = None
            else:
                for ticker in batch:
                    try:
                        df_t = yf.download(ticker, start=start, progress=False, actions=True)
                        result[ticker] = df_t if not df_t.empty else None
                    except Exception:
                        result[ticker] = None
            return result
        except Exception as e:
            last_exc = e
            logging.warning("Erro no yfinance.download (attempt %d): %s", attempt+1, str(e))
            attempt += 1
            time.sleep(2 ** attempt)  # backoff exponencial
    logging.error("Todas tentativas falharam para batch (%s). Último erro: %s", joined, last_exc)
    # fallback: tentar baixar ticker a ticker
    result = {}
    for ticker in batch:
        try:
            df_t = yf.download(ticker, start=start, progress=False, actions=True)
            result[ticker] = df_t if not df_t.empty else None
        except Exception as e:
            logging.warning("Fallback individual falhou para %s: %s", ticker, e)
            result[ticker] = None
    return result

def download_all_histories(tickers: List[str], start: str = DEFAULT_START, force: bool = False, save_summary: bool = True, save_csv_per_ticker: bool = True):
    """
    Processo principal: recebe lista de tickers (strings), baixa históricos e salva parquet + csv por ticker.
    - force: se True, re-baixa mesmo que arquivo exista.
    - save_csv_per_ticker: se True salva um CSV para cada ticker (além do parquet).
    - retorna um DataFrame resumo com status por ticker.
    """
    os.makedirs(HIST_DIR, exist_ok=True)
    tickers = [t for t in tickers if isinstance(t, str) and t.strip()]
    tickers = list(dict.fromkeys(tickers))
    summary = []
    for i in range(0, len(tickers), BATCH_SIZE):
        batch = tickers[i:i+BATCH_SIZE]
        to_download = [t for t in batch if force or not ticker_exists_local(t)]
        if not to_download:
            logging.info("Batch %d: todos já existem localmente — pulando.", i//BATCH_SIZE+1)
            for t in batch:
                summary.append({
                    "ticker": t,
                    "status": "skipped_local",
                    "rows": None,
                    "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet") if ticker_exists_local(t) else None,
                    "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if os.path.exists(os.path.join(HIST_DIR, f"{t}.csv")) else None
                })
            continue

        logging.info("Processando batch %d/%d (download %d/%d)", i//BATCH_SIZE+1, (len(tickers)+BATCH_SIZE-1)//BATCH_SIZE, len(to_download), len(batch))
        results = download_batch(to_download, start=start)
        for t in batch:
            df_t = results.get(t) if t in results else None
            if df_t is None or (isinstance(df_t, pd.DataFrame) and df_t.empty):
                logging.warning("Nenhum dado para %s em batch; tentativa isolada...", t)
                try:
                    single = yf.download(t, start=start, progress=False, actions=True)
                    df_t = single if not single.empty else None
                except Exception:
                    df_t = None
            if df_t is None or df_t.empty:
                logging.error("Falha obtendo dados para %s", t)
                summary.append({"ticker": t, "status": "failed", "rows": 0, "saved_parquet": None, "saved_csv": None})
            else:
                try:
                    save_history_df(t, df_t, save_csv=save_csv_per_ticker)
                    summary.append({
                        "ticker": t,
                        "status": "ok",
                        "rows": len(df_t),
                        "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet"),
                        "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if save_csv_per_ticker else None
                    })
                except Exception as e:
                    logging.exception("Erro salvando para %s: %s", t, e)
                    summary.append({"ticker": t, "status": "save_error", "rows": len(df_t) if isinstance(df_t, pd.DataFrame) else None, "saved_parquet": None, "saved_csv": None})
            time.sleep(SLEEP_BETWEEN_TICKERS)
        time.sleep(SLEEP_BETWEEN_BATCHES)

    df_summary = pd.DataFrame(summary)
    if save_summary:
        ts = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        summary_path = os.path.join(HIST_DIR, f"download_summary_{ts}.csv")
        df_summary.to_csv(summary_path, index=False)
        logging.info("Resumo salvo em %s", summary_path)
    return df_summary

def combine_all_to_single_parquet(out_path: str = os.path.join(HIST_DIR, "all_histories.parquet"), out_csv: Optional[str] = os.path.join(HIST_DIR, "all_histories.csv"), tickers: Optional[List[str]] = None):
    """
    Lê todos os parquets em HIST_DIR (ou tickers list) e concatena em formato long:
    columns: ['ticker','date', 'Open','High','Low','Close','Adj Close','Volume', 'Dividends','Stock Splits']
    Salva em parquet e opcionalmente em csv.
    """
    files = []
    if tickers:
        files = [os.path.join(HIST_DIR, f"{t}.parquet") for t in tickers if os.path.exists(os.path.join(HIST_DIR, f"{t}.parquet"))]
    else:
        files = [os.path.join(HIST_DIR, f) for f in os.listdir(HIST_DIR) if f.endswith(".parquet")]
    dfs = []
    for f in files:
        try:
            df = pd.read_parquet(f)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
            fname = os.path.basename(f).replace(".parquet","")
            if 'ticker' not in df.columns:
                df.insert(0, 'ticker', fname)
            dfs.append(df)
        except Exception as e:
            logging.warning("Erro lendo %s: %s", f, e)
    if not dfs:
        raise RuntimeError("Nenhum parquet encontrado para combinar.")
    big = pd.concat(dfs, ignore_index=True, sort=False)
    big.to_parquet(out_path, index=False)
    logging.info("Combined saved to %s (rows=%d)", out_path, len(big))
    if out_csv:
        try:
            # converter date para formato iso ao salvar CSV
            if 'date' in big.columns:
                big['date'] = pd.to_datetime(big['date']).dt.strftime('%Y-%m-%d')
            big.to_csv(out_csv, index=False)
            logging.info("Combined CSV saved to %s", out_csv)
        except Exception as e:
            logging.exception("Erro salvando combined CSV: %s", e)
    return big

if __name__ == "__main__":
    # 1) carregue a lista de tickers a partir do arquivo que você já salvou
    # Usa caminho relativo para acessar a pasta data no mesmo nível de data_sources
    tickers_file = os.path.join("..", "data", "tickers_ibrx100_full.csv")
    if os.path.exists(tickers_file):
        df = pd.read_csv(tickers_file)
        if 'Ticker' in df.columns:
            tickers = df['Ticker'].dropna().astype(str).tolist()
        else:
            tickers = df.iloc[:,0].dropna().astype(str).tolist()
    else:
        raise RuntimeError(f"Não encontrou {tickers_file}. Coloque seu CSV de tickers na pasta 'aurum/data/' ou edite este script.")

    # 2) opção: validar/normalizar tickers (garantir sufixo .SA)
    def normalize(t):
        t = str(t).strip().upper()
        if not t.endswith(".SA"):
            t = t.replace(".SA","") + ".SA"
        return t
    tickers = [normalize(t) for t in tickers]
    print("Tickers a baixar:", len(tickers), tickers[:10])

    # 3) executar (force=True re-baixa mesmo se já existir)
    summary_df = download_all_histories(tickers, start=DEFAULT_START, force=False, save_summary=True, save_csv_per_ticker=True)
    print(summary_df.head(50))

    # 4) opcional: combinar tudo em um único parquet e CSV (pode ser grande)
    combined = combine_all_to_single_parquet()
    print("Combined rows:", len(combined))


In [ ]:
import pandas as pd
import os
import io

def print_header(title):
    """
    Função auxiliar para imprimir um cabeçalho formatado no log.
    """
    print("\n" + "=" * 70)
    print(f" {title.upper()} ")
    print("=" * 70)

def analyze_dataframe(file_path):
    """
    Carrega e analisa um DataFrame de histórico de ações.
    """
    
    print(f"--- Iniciando Análise do Arquivo: {file_path} ---")

    # --- 1. Verificação e Carregamento dos Dados ---
    
    if not os.path.exists(file_path):
        print_header("[ERRO] Arquivo não encontrado")
        print(f"O arquivo no caminho '{file_path}' não foi localizado.")
        print("Por favor, verifique se o caminho está correto e a pasta 'historical' existe.")
        print("=" * 70)
        return

    try:
        # Tenta carregar o CSV. 
        # A coluna 'date' é convertida para datetime no carregamento.
        df = pd.read_csv(file_path, parse_dates=['date'])
        print(f"\n[SUCESSO] Arquivo carregado. Total de {len(df)} linhas e {len(df.columns)} colunas.")
    except Exception as e:
        print_header("[ERRO] Falha ao carregar o arquivo")
        print(f"Ocorreu um erro ao tentar ler o arquivo CSV: {e}")
        print("=" * 70)
        return

    # --- 2. Amostra dos Dados (Head) ---
    print_header("1. Amostra dos Dados (Primeiras 5 Linhas)")
    # .to_string() formata o DataFrame como uma tabela de texto legível
    print(df.head().to_string())

    # --- 3. Informações do DataFrame (Info) ---
    print_header("2. Informações do DataFrame (Tipos de Coluna e Nulos)")
    # O df.info() imprime diretamente. Para capturá-lo e formatá-lo,
    # usamos um buffer de string.
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()
    print(info_str)

    # --- 4. Contagem de Valores Nulos ---
    print_header("3. Resumo de Valores Nulos por Coluna")
    null_counts = df.isnull().sum()
    
    if null_counts.sum() == 0:
        print("Ótimo! Não há valores nulos em nenhuma coluna.")
    else:
        # Filtra para mostrar apenas colunas que *possuem* valores nulos
        print(null_counts[null_counts > 0].to_string())

    # --- 5. Estatísticas Descritivas ---
    print_header("4. Estatísticas Descritivas (Colunas Numéricas)")
    # O 'include='np.number' garante que só analisará colunas numéricas
    # .to_string() formata a saída para melhor visualização no log
    try:
        print(df.describe().to_string())
    except Exception as e:
        print(f"Não foi possível calcular estatísticas descritivas: {e}")

    # --- 6. Análise de Tickers ---
    print_header("5. Análise da Coluna 'ticker'")
    if 'ticker' in df.columns:
        unique_tickers = df['ticker'].unique()
        num_unique_tickers = len(unique_tickers)
        print(f"Total de tickers únicos encontrados: {num_unique_tickers}")
        
        # Mostra uma amostra se houver muitos tickers
        if num_unique_tickers > 10:
            print(f"Amostra de tickers: {unique_tickers[:10]}...")
        else:
            print(f"Tickers presentes: {unique_tickers}")
    else:
        print("Coluna 'ticker' não encontrada.")

    print("\n" + "=" * 70)
    print("--- ANÁLISE CONCLUÍDA ---")
    print("=" * 70)

# --- Ponto de Execução Principal ---
if __name__ == "__main__":
    
    # Define o caminho do arquivo
    path_do_arquivo = 'data/historical/all_histories.csv'
    
    # Executa a função de análise
    analyze_dataframe(path_do_arquivo)

In [ ]:
import pandas as pd
import numpy as np
import logging
import os # Garante que 'os' está importado

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- 1. Carregar os Dados ---
try:
    # Tenta carregar o Parquet primeiro, é mais rápido
    df_prices_raw = pd.read_parquet("data/historical/all_histories.parquet")
    logger.info(f"Dados de preço brutos carregados do PARQUET: {df_prices_raw.shape[0]} linhas")
except FileNotFoundError:
    logger.warning("Arquivo all_histories.parquet não encontrado! Tentando carregar o CSV...")
    try:
        # Carrega o CSV como fallback
        df_prices_raw = pd.read_csv("data/historical/all_histories.csv", parse_dates=['date'])
        logger.info(f"Dados de preço brutos carregados do CSV: {df_prices_raw.shape[0]} linhas")
    except FileNotFoundError as e:
        logger.error("Nenhum arquivo de histórico (parquet ou csv) encontrado. Execute o download primeiro.")
        raise e
except Exception as e:
    logger.error(f"Erro ao carregar dados: {e}")
    raise

# --- 2. Limpeza Crítica (Remover NaN de pré-IPO) ---
# Remove todas as linhas onde a ação ainda não existia (Adj Close ou Volume são NaN)
df_prices_clean = df_prices_raw.dropna(subset=['Adj Close', 'Volume'])

# --- 3. Limpeza Opcional (Remover dias sem negociação) ---
df_prices_clean = df_prices_clean[df_prices_clean['Volume'] > 0]

linhas_removidas = len(df_prices_raw) - len(df_prices_clean)
logger.info(f"Dados de preço limpos: {df_prices_clean.shape[0]} linhas (removidas {linhas_removidas} linhas com NaN ou Volume 0)")

# --- 4. Salvar os Arquivos Limpos ---
# Garante que o diretório existe
output_dir = "data/historical"
os.makedirs(output_dir, exist_ok=True)

parquet_path = os.path.join(output_dir, "all_histories_cleaned.parquet")
csv_path = os.path.join(output_dir, "all_histories_cleaned.csv")

try:
    # Salvar em Parquet (preferencial para o próximo passo)
    df_prices_clean.to_parquet(parquet_path, index=False)
    logger.info(f"✅ Arquivo limpo salvo em (Parquet): {parquet_path}")

    # Salvar em CSV (para sua verificação)
    # date_format garante que a data seja salva em formato legível
    df_prices_clean.to_csv(csv_path, index=False, date_format='%Y-%m-%d')
    logger.info(f"✅ Arquivo limpo salvo em (CSV): {csv_path}")

except Exception as e:
    logger.error(f"❌ Erro ao salvar arquivos limpos: {e}")

print(f"\nLimpeza concluída. {linhas_removidas} linhas de 'lookahead' removidas.")
print(df_prices_clean.head())

In [ ]:
import pandas as pd
import numpy as np
import logging
import os

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

INPUT_FILE = "data/historical/all_histories_cleaned.parquet"
OUTPUT_DIR = "data/historical"

def gerar_precos_pivotados():
    logger.info("Iniciando o Passo 1: Geração dos Preços Pivotados (Wide)...")
    
    # 1. Carregar os dados de preço limpos
    try:
        df_prices_clean = pd.read_parquet(INPUT_FILE)
        logger.info(f"Dados limpos '{INPUT_FILE}' carregados.")
    except FileNotFoundError:
        logger.error(f"ARQUIVO NÃO ENCONTRADO: {INPUT_FILE}")
        logger.error("Execute o 'Script 1: Limpeza' primeiro.")
        return

    df_prices_clean['date'] = pd.to_datetime(df_prices_clean['date'])

    # 2. Criar a base de preços mensais
    logger.info("Reamostrando para Frequência Mensal (M e MS)...")
    # 2a. Fechamento (Último dia do mês 'M')
    df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()

    # 2b. Abertura (Primeiro dia do mês 'MS')
    df_open_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('MS').first()

    # 3. Pivotar para o formato "wide"
    logger.info("Corrigindo MultiIndex e Pivotando...")
    
    # CORREÇÃO: Dropar a coluna 'ticker' duplicada antes de resetar o índice
    df_prices_mensal_long = df_prices_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()
    df_open_mensal_long = df_open_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()

    # Agora o .pivot() funcionará
    df_close_wide = df_prices_mensal_long.pivot(index='date', columns='ticker', values='Adj Close')
    df_open_wide = df_open_mensal_long.pivot(index='date', columns='ticker', values='Open')

    # 4. Sincronizar os índices de data
    logger.info("Sincronizando e preenchendo índices de data...")
    idx_union = df_close_wide.index.union(df_open_wide.index)
    
    df_close_wide = df_close_wide.reindex(idx_union, method='ffill')
    df_open_wide = df_open_wide.reindex(idx_union, method='ffill')

    # Preenche NaNs (pré-IPO e pós-delist)
    df_close_wide = df_close_wide.ffill().bfill()
    df_open_wide = df_open_wide.ffill().bfill()

    # --- 5. SALVAR OS ARQUIVOS FALTOSOS ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    path_close = os.path.join(OUTPUT_DIR, "prices_close_wide.parquet")
    path_open = os.path.join(OUTPUT_DIR, "prices_open_wide.parquet")
    
    df_close_wide.to_parquet(path_close)
    df_open_wide.to_parquet(path_open)
    
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_close}")
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_open}")
    print("\n--- Amostra de Fechamento (Wide) ---")
    print(df_close_wide.tail())

if __name__ == "__main__":
    gerar_precos_pivotados()

In [ ]:
import pandas as pd
import numpy as np
import os
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("Iniciando Passo 2: Cálculo da Volatilidade...")

# Carregar os dados de preço DIÁRIOS (limpos)
df_prices_daily = pd.read_parquet("data/historical/all_histories_cleaned.parquet")
df_prices_daily['date'] = pd.to_datetime(df_prices_daily['date'])

# 1. Calcular retornos diários
df_prices_daily['returns'] = df_prices_daily.groupby('ticker')['Adj Close'].pct_change()

# 2. Calcular a volatilidade móvel de 63 dias (~3 meses de negociação)
# .std() calcula o desvio padrão (volatilidade)
# reset_index(0, drop=True) é necessário após o .rolling() em um groupby
df_prices_daily['VOLATILIDADE'] = df_prices_daily.groupby('ticker')['returns'].rolling(window=63).std().reset_index(0, drop=True)

# 3. Resample da volatilidade para MENSAL (pegamos o último valor do mês)
df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()

logger.info(f"✅ Volatilidade mensal calculada. {len(df_vol_mensal)} registros.")
print(df_vol_mensal.tail())